<a href="https://colab.research.google.com/github/Farheen-Qazi/Data_Science_And_Structures/blob/main/20SW055(LAB_15).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Recommendations with MovieTweetings: Most Popular Recommendation

Now that you have created the necessary columns we will be using throughout the rest of the lesson on creating recommendations, let's get started with the first of our recommendations.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('/content/movies_clean.csv')
reviews = pd.read_csv('/content/reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [ ]:
def popular_recommendations(user_id, n_top):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Do stuff
    # calculate average rating
    result = reviews.groupby(['movie_id'])['rating'].mean()
    result = pd.DataFrame(result)

    # calculate review count
    result['review_count'] = reviews.groupby(['movie_id'])['rating'].count()

    # drop movies having less than 5 rating
    result = result[int(result.review_count) >= 5]

    # get rating by most recent
    result['recent_date'] = reviews.groupby(['movie_id'])['date'].max()
    result = result.merge(movies[['movie_id','movie']],on='movie_id')

    # sort the values
    result = result.sort_values(['rating','review_count','recent_date'],ascending=False)
    top_movies = result[0:n_top]['movie'].tolist()

    return top_movies # a list of the n_top movies as recommended

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [ ]:
# Put your solutions for each of the cases here

# Top 20 movies recommended for id 1

recs_20_for_1 =popular_recommendations('1', int(20))

# Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations('53968', 5)

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations('70000', 100)

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations('43', 35)



TypeError: ignored

In [ ]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

# check 1
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same?

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```

In [ ]:
def popular_recommendations_filtered(movies, reviews,user_id, n_top, years, genres):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    year - a list of movie years
    genres - a list of genres
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Do stuff
    # calculate average rating
    result = reviews.groupby(['movie_id'])['rating'].mean()
    result = pd.DataFrame(result)

    # calculate review count
    result['review_count'] = reviews.groupby(['movie_id'])['rating'].count()

    # drop movies having less than 5 rating
    result = result[result.review_count >= 5]

    # get rating by most recent
    result['recent_date'] = reviews.groupby(['movie_id'])['date'].max()

    result = result.merge(movies[['movie_id','movie','date']+genres],on='movie_id')
    result = result[result.date.isin(years)]
    # filter movie by genre
    try:
        result = result[result[genres].eq(1).any(1)]
    except:
        pass
    # sort the values
    result = result.sort_values(['rating','review_count','recent_date'],ascending=False)
    top_movies = result[0:n_top]['movie'].tolist()

    return top_movies # a list of the n_top movies as recommended

In [ ]:
top20 = popular_recommendations_filtered(movies, reviews,'1', 20,years=['2015', '2016', '2017', '2018'], genres=['History'] )

In [ ]:
assert top20 == t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])